# Tutorial 8 - Options

Please complete this tutorial to get an overview of options and an implementation of SMDP Q-Learning and Intra-Option Q-Learning.


### References:

 [Recent Advances in Hierarchical Reinforcement
Learning](https://people.cs.umass.edu/~mahadeva/papers/hrl.pdf) is a strong recommendation for topics in HRL that was covered in class. Watch Prof. Ravi's lectures on moodle or nptel for further understanding the core concepts. Contact the TAs for further resources if needed.


In [1]:
!pip3 install gym==0.15.7
!pip3 install numpy==1.23.1

In [7]:
'''
A bunch of imports, you don't have to worry about these
'''

import numpy as np
from tqdm import tqdm
import random
import gym
# from gym.wrappers import Monitor
import glob
import io
import matplotlib.pyplot as plt
from IPython.display import HTML

In [8]:
'''
The environment used here is extremely similar to the openai gym ones.
At first glance it might look slightly different.
The usual commands we use for our experiments are added to this cell to aid you
work using this environment.
'''

#Setting up the environment
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv
env = CliffWalkingEnv()

env.reset()

#Current State
print(env.s)

# 4x12 grid = 48 states
print ("Number of states:", env.nS)

# Primitive Actions
action = ["up", "right", "down", "left"]
#correspond to [0,1,2,3] that's actually passed to the environment

# either go left, up, down or right
print ("Number of actions that an agent can take:", env.nA)

# Example Transitions
rnd_action = random.randint(0, 3)
print ("Action taken:", action[rnd_action])
next_state, reward, is_terminal, t_prob= env.step(rnd_action)
print ("Transition probability:", t_prob)
print ("Next state:", next_state)
print ("Reward recieved:", reward)
print ("Terminal state:", is_terminal)
env.render()

36
Number of states: 48
Number of actions that an agent can take: 4
Action taken: left
Transition probability: {'prob': 1.0}
Next state: 36
Reward recieved: -1
Terminal state: False
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T



#### Options
We custom define very simple options here. They might not be the logical options for this settings deliberately chosen to visualise the Q Table better.


In [9]:
# We are defining two more options here
# Option 1 ["Away"] - > Away from Cliff (ie keep going up)
# Option 2 ["Close"] - > Close to Cliff (ie keep going down)

def Away(env,state):

    optdone = False
    optact = 0

    if (int(state/12) == 0):
        optdone = True

    return [optact,optdone]

def Close(env,state):

    optdone = False
    optact = 2

    if (int(state/12) == 2):
        optdone = True

    if (int(state/12) == 3):
        optdone = True

    return [optact,optdone]


'''
Now the new action space will contain
Primitive Actions: ["up", "right", "down", "left"]
Options: ["Away","Close"]
Total Actions :["up", "right", "down", "left", "Away", "Close"]
Corresponding to [0,1,2,3,4,5]
'''

'\nNow the new action space will contain\nPrimitive Actions: ["up", "right", "down", "left"]\nOptions: ["Away","Close"]\nTotal Actions :["up", "right", "down", "left", "Away", "Close"]\nCorresponding to [0,1,2,3,4,5]\n'

# Task 1
Complete the code cell below


In [10]:
#Q-Table: (States x Actions) === (env.ns(48) x total actions(6))

q_values_SMDP2 = np.zeros((48,6))
#Update_Frequency Data structure? Check TODO 4


ufd2 = np.zeros((48,6))#Update_Frequency Data structure

actions=[0,1,2,3,4,5]
# TODO: epsilon-greedy action selection function
seed = 36
rg = np.random.RandomState(seed)

def egreedy_policy(q_values,state,epsilon):
    if rg.rand() < epsilon:
        return rg.choice([0,1,2,3,4,5])
    else:
        return np.argmax(q_values[state])

# Task 2
Below is an incomplete code cell with the flow of SMDP Q-Learning. Complete the cell and train the agent using SMDP Q-Learning algorithm.
Keep the **final Q-table** and **Update Frequency** table handy (You'll need it in TODO 4)

In [14]:
#### SMDP Q-Learning

# Add parameters you might need here
gamma = 0.9
alpha = 0.4
q_values_SMDP = np.zeros((48,6))
ufd1 = np.zeros((48,6))#Update_Frequency Data structure
# Iterate over 1000 episodes
for _ in range(1000):
    state = env.reset()
    done = False

    # While episode is not over
    while not done:

        # Choose action
        action = egreedy_policy(q_values_SMDP, state, epsilon=0.1)

        # Checking if primitive action
        if action < 4:
            # Perform regular Q-Learning update for state-action pair

            next_state, reward, done,_= env.step(action)
            q_values_SMDP[state, action] += alpha*(reward + gamma*np.max([q_values_SMDP[next_state, action] for action in actions]) - q_values_SMDP[state, action])
            ufd1[state,action] += 1
            state = next_state

        # Checking if action chosen is an option
        reward_bar = 0
        if action == 4: # action => Away option

            initial_state = np.copy(state)
            optdone = False
            count=0
            while (optdone == False):

                # Think about what this function might do?
                optact,_ = Away(env,state)
                next_state, reward, done,_ = env.step(optact)


                _,optdone = Away(env,next_state)

                # Is this formulation right? What is this term?
                # Ans: the accumulates return for the entire option
                reward_bar = reward_bar +  (gamma**count)*reward
                count+=1
                # Complete SMDP Q-Learning Update
                # Remember SMDP Updates. When & What do you update?
                state = next_state

            q_values_SMDP[initial_state, action] += alpha*(reward_bar + (gamma**count)*np.max([q_values_SMDP[state, action] for action in actions]) - q_values_SMDP[initial_state, action])
            ufd1[initial_state,action] += 1


        if action == 5: # action => Close option

            initial_state = np.copy(state)
            optdone = False
            count=0
            while (optdone == False):

                # Think about what this function might do?
                optact,_ = Close(env,state)
                next_state, reward, done,_ = env.step(optact)


                _,optdone = Close(env,next_state)

                # Is this formulation right? What is this term?
                # Ans: the accumulates return for the entire option
                reward_bar = reward_bar +  (gamma**count)*reward
                count+=1
                # Complete SMDP Q-Learning Update
                # Remember SMDP Updates. When & What do you update?
                state = next_state

            q_values_SMDP[initial_state, action] += alpha*(reward_bar + (gamma**count)*np.max([q_values_SMDP[state, action] for action in actions]) - q_values_SMDP[initial_state, action])
            ufd1[initial_state,action] += 1



# Task 3
Using the same options and the SMDP code, implement Intra Option Q-Learning (In the code cell below). You *might not* always have to search through options to find the options with similar policies, think about it. Keep the **final Q-table** and **Update Frequency** table handy (You'll need it in TODO 4)



In [16]:
#### Intra-Option Q-Learning



# Add parameters you might need here
gamma = 0.9
alpha = 0.4

# Iterate over 1000 episodes
for _ in range(1000):
    state = env.reset()
    done = False

    # While episode is not over
    while not done:

        # Choose action
        action = egreedy_policy(q_values_SMDP2, state, epsilon=0.1)

        # Checking if primitive action
        if action < 4:
            # Perform regular Q-Learning update for state-action pair

            next_state, reward, done,_ = env.step(action)
            q_values_SMDP2[state, action] += alpha*(reward + gamma*np.max([q_values_SMDP2[next_state, action] for action in actions]) - q_values_SMDP2[state, action])
            ufd2[state,action] += 1

            state = next_state

        # Checking if action chosen is an option
        reward_bar = 0
        if action == 4: # action => Away option

            #initial_state = state
            optdone = False
            #count=0
            while (optdone == False) :

                # Think about what this function might do?
                optact,_ = Away(env,state)
                #
                next_state, reward, done,_ = env.step(optact)
                _,optdone = Away(env,next_state)

                q_values_SMDP2[state, optact] += alpha*(reward + gamma*np.max([q_values_SMDP2[next_state, action] for action in actions]) - q_values_SMDP2[state, optact])
                ufd2[state,optact] += 1

                if not optdone:
                  q_values_SMDP2[state, action] += alpha*(reward + gamma*q_values_SMDP2[next_state, action] - q_values_SMDP2[state, action])
                  ufd2[state,action] += 1
                else:
                  q_values_SMDP2[state, action] += alpha*(reward + gamma*np.max([q_values_SMDP2[next_state, action] for action in actions]) - q_values_SMDP2[state, action])
                  ufd2[state,action] += 1


                # Complete SMDP Q-Learning Update
                # Remember SMDP Updates. When & What do you update?
                state = next_state





        if action == 5: # action => Close option

            #initial_state = state
            optdone = False
            #count=0
            while (optdone == False) :

                # Think about what this function might do?
                optact,_ = Close(env,state)
                next_state, reward, done,_ = env.step(optact)
                _,optdone = Close(env,next_state)

                q_values_SMDP2[state, optact] += alpha*(reward + gamma*np.max([q_values_SMDP2[next_state, action] for action in actions]) - q_values_SMDP2[state, optact])
                ufd2[state,optact] += 1

                if not optdone:
                  q_values_SMDP2[state, action] += alpha*(reward + gamma*q_values_SMDP2[next_state, action] - q_values_SMDP2[state, action])
                  ufd2[state,action] += 1
                else:
                  q_values_SMDP2[state, action] += alpha*(reward + gamma*np.max([q_values_SMDP2[next_state, action] for action in actions]) - q_values_SMDP2[state, action])
                  ufd2[state,action] += 1


                # Complete SMDP Q-Learning Update
                # Remember SMDP Updates. When & What do you update?
                state = next_state

# Task 4
Compare the two Q-Tables and Update Frequencies and provide comments.

In [17]:
from pandas import DataFrame
def table_render(arr):
  print(DataFrame(arr,columns=["up", "right", "down", "left", "Away", "Close"]))

table_render(q_values_SMDP)

          up       right        down      left      Away       Close
0  -7.729438   -7.685514   -7.681422 -7.699484 -7.693760   -7.682491
1  -7.519931   -7.439467   -7.436725 -7.479699 -7.514018   -7.444857
2  -7.201298   -7.161441   -7.166289 -7.365133 -7.250717   -7.165592
3  -7.060948   -6.849887   -6.851390 -7.016437 -6.891528   -6.855066
4  -6.609679   -6.503400   -6.504559 -6.765017 -6.648498   -6.502854
5  -6.176723   -6.117531   -6.119634 -6.537352 -6.312034   -6.120239
6  -5.954649   -5.689220   -5.691414 -6.049263 -5.854036   -5.690576
7  -5.395808   -5.213450   -5.214202 -5.433794 -5.389702   -5.213467
8  -4.795971   -4.684000   -4.684049 -5.009996 -5.007942   -4.683748
9  -4.479651   -4.094307   -4.094473 -4.394478 -4.273077   -4.094391
10 -3.594209   -3.438714   -3.438715 -4.177258 -3.611374   -3.438828
11 -3.195192   -3.047161   -2.709963 -3.561978 -2.786104   -2.709976
12 -7.662038   -7.444197   -7.449470 -7.526093 -7.595727   -7.447391
13 -7.277369   -7.171653   -7.1712

In [19]:
table_render(q_values_SMDP2)

          up       right        down      left      Away       Close
0  -7.792267   -7.709757   -7.710474 -7.791814 -7.791814   -7.709723
1  -7.647197   -7.457365   -7.457592 -7.482925 -7.646183   -7.457277
2  -7.387604   -7.175244   -7.175535 -7.500283 -7.349866   -7.175349
3  -7.018157   -6.861658   -6.861833 -6.887870 -7.007155   -6.861724
4  -6.540943   -6.513052   -6.513181 -6.881067 -6.540192   -6.513109
5  -6.398230   -6.125691   -6.125710 -6.523192 -6.392955   -6.125699
6  -6.044907   -5.695276   -5.695315 -6.072767 -6.043893   -5.695288
7  -5.393733   -5.217012   -5.217022 -5.421611 -5.391594   -5.217016
8  -5.078640   -4.685583   -4.685585 -5.056706 -4.984866   -4.685582
9  -4.498153   -4.095098   -4.095099 -4.927027 -4.490912   -4.095098
10 -3.994070   -3.439000   -3.439000 -4.270104 -3.617607   -3.438999
11 -2.902852   -2.786104   -2.710000 -2.949415 -2.786104   -2.710000
12 -7.936144   -7.457883   -7.458061 -7.546898 -7.936144   -7.458061
13 -7.710566   -7.175695   -7.1757

Note that both the methods have converged to similar Q-values.
 The q-values are very low; close to -106 for action 'down' and option 'close' in states 25-35, since it represents the row above the cliff, and the agent has learnt to avoid those actions.

In [20]:
table_render(ufd1)

        up   right   down  left  Away  Close
0     37.0    90.0   59.0  37.0  36.0   20.0
1     35.0    89.0   56.0  24.0  35.0   22.0
2     32.0    98.0   54.0  23.0  32.0   22.0
3     32.0    97.0   50.0  21.0  29.0   22.0
4     27.0   100.0   46.0  19.0  28.0   20.0
5     24.0    88.0   44.0  19.0  25.0   20.0
6     23.0    79.0   42.0  16.0  22.0   19.0
7     19.0    71.0   37.0  13.0  19.0   19.0
8     16.0    61.0   35.0  12.0  18.0   19.0
9     15.0    55.0   34.0  10.0  14.0   19.0
10    11.0    34.0   31.0   9.0  11.0   21.0
11    10.0     9.0   31.0   8.0   8.0   24.0
12    29.0    73.0   27.0  35.0  28.0   27.0
13    24.0    88.0   29.0  23.0  25.0   29.0
14    23.0    94.0   31.0  21.0  25.0   30.0
15    21.0    92.0   30.0  19.0  21.0   29.0
16    18.0    91.0   30.0  16.0  19.0   29.0
17    17.0    88.0   29.0  16.0  16.0   29.0
18    15.0    84.0   30.0  14.0  14.0   27.0
19    13.0    74.0   28.0  13.0  13.0   27.0
20    12.0    64.0   27.0  10.0  12.0   27.0
21    10.0

In [21]:
table_render(ufd2)

        up   right    down  left  Away  Close
0     39.0    69.0    51.0  37.0  37.0   36.0
1     42.0    85.0    51.0  25.0  37.0   36.0
2     36.0    87.0    51.0  24.0  33.0   38.0
3     36.0    93.0    51.0  21.0  30.0   40.0
4     31.0    93.0    50.0  20.0  26.0   39.0
5     29.0    87.0    45.0  19.0  26.0   36.0
6     28.0    83.0    45.0  17.0  24.0   36.0
7     23.0    83.0    43.0  14.0  19.0   36.0
8     22.0    76.0    40.0  12.0  18.0   35.0
9     19.0    64.0    42.0  12.0  16.0   35.0
10    17.0    41.0    44.0  10.0  11.0   36.0
11    10.0     8.0    57.0   7.0   8.0   52.0
12    95.0    70.0    44.0  35.0  94.0   43.0
13    68.0    81.0    56.0  24.0  65.0   52.0
14    56.0    97.0    59.0  21.0  53.0   58.0
15    58.0   101.0    58.0  18.0  52.0   55.0
16    51.0   103.0    59.0  19.0  48.0   56.0
17    41.0   101.0    61.0  15.0  40.0   58.0
18    44.0    93.0    60.0  13.0  39.0   56.0
19    45.0    90.0    59.0  14.0  42.0   55.0
20    33.0    81.0    59.0  12.0  

In [22]:
np.sum(ufd1),np.sum(ufd2)

(21188.0, 23541.0)

In [23]:
print(["up", "right", "down", "left", "Away", "Close"])
print(np.sum(ufd1,axis=0))
print(np.sum(ufd2,axis=0))

['up', 'right', 'down', 'left', 'Away', 'Close']
[ 2509. 13858.  2098.   871.   850.  1002.]
[ 3136. 13766.  2884.   867.  1354.  1534.]


The frequency of updates in intra-option Q-learning surpasses that of SMDP Q-learning. Pay particular attention to the occurrences of actions such as 'up' and 'down', and options like 'Away' and 'Close'; we will observe a notably higher frequency in intra-option Q-learning, as expected. This disparity arises because in intra-option Q-learning, actions 'up' and 'down' are updated even while executing options, and options themselves are updated at each intermediate step.